<a href="https://colab.research.google.com/github/Shikhar22255/assignment-AI-Developer-Exercise/blob/main/assignment_AI_Developer_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install playsound

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=a4df0fd6c02a6216f3a3ad893952e983e237bc764d51ee8056035e14ce9fd767
  Stored in directory: /root/.cache/pip/wheels/50/98/42/62753a9e1fb97579a0ce2f84f7db4c21c09d03bb2091e6cef4
Successfully built playsound


In [4]:
pip install gtts

In [3]:
pip install speechrecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 42.6 MB/s eta 0:00:00


In [2]:
pip install openai

In [19]:
! pip install --upgrade openai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 26.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1


In [23]:
import os
import time
import json
import requests
import speech_recognition as sr
from gtts import gTTS
import playsound

# ✅ Securely Fetch API Key from Environment Variable
API_KEY = os.getenv("OPENROUTER_API_KEY", "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")
API_URL = "https://openrouter.ai/api/v1/chat/completions"

# ✅ Load or Initialize User Progress
def load_progress():
    if os.path.exists("progress.json"):
        with open("progress.json", "r") as file:
            return json.load(file)
    return {}

def save_progress(data):
    with open("progress.json", "w") as file:
        json.dump(data, file, indent=4)

progress = load_progress()

def chat_with_ai(user_input):
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    data = {
        "model": "mistralai/mistral-7b-instruct",  # ✅ Correct model for OpenRouter
        "messages": [{"role": "system", "content": "You are a verbal communication coach."},
                     {"role": "user", "content": user_input}]
    }
    retries, delay = 3, 5
    while retries > 0:
        try:
            response = requests.post(API_URL, headers=headers, json=data)
            response_json = response.json()

            # ✅ Debugging: Print Full Response
            print("🔍 API Response:", response_json)

            # ✅ Check if 'choices' exists
            if "choices" in response_json and len(response_json["choices"]) > 0:
                return response_json["choices"][0]["message"]["content"]
            else:
                return f"❌ Unexpected API Response: {response_json}"

        except requests.exceptions.RequestException as e:
            print(f"⚠️ API Error: {e}, Retrying in {delay} seconds...")
            time.sleep(delay)
            delay *= 2
            retries -= 1
    return "❌ No available models found or request failed."

def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Speak now...")
        audio = recognizer.listen(source)
    try:
        text = recognizer.recognize_google(audio)
        print("You said:", text)
        return text
    except sr.UnknownValueError:
        return "Sorry, I could not understand."
    except sr.RequestError:
        return "Error with speech recognition service."

def text_to_speech(text):
    try:
        tts = gTTS(text)
        tts.save("response.mp3")
        playsound.playsound("response.mp3")
    except Exception as e:
        print(f"⚠️ Text-to-Speech Error: {e}")

def evaluate_presentation(script):
    prompt = f"Evaluate this presentation script:\n{script}\nProvide feedback on structure, tone, clarity, and engagement."
    feedback = chat_with_ai(prompt)
    return feedback

def training_module(module_type):
    prompts = {
        "impromptu": "Explain why teamwork is important in 60 seconds.",
        "storytelling": "Tell a short story about overcoming a challenge.",
        "conflict_resolution": "Respond diplomatically to: 'I'm upset because you missed a deadline.'"
    }
    if module_type not in prompts:
        return "Invalid training module."
    user_response = input(f"{prompts[module_type]}\nYour Response: ")
    feedback = chat_with_ai(user_response)
    progress[module_type] = {"response": user_response, "feedback": feedback}
    save_progress(progress)
    return feedback

def main():
    while True:
        print("\nOptions:\n1. Chat with AI\n2. Voice Input (May not work in all environments)\n3. Evaluate Presentation\n4. Training Modules\n5. Exit")
        choice = input("Choose an option: ")

        if choice == "1":
            user_input = input("Enter your message: ")
            response = chat_with_ai(user_input)
            print("AI:", response)
            text_to_speech(response)

        elif choice == "2":
            user_input = speech_to_text()
            response = chat_with_ai(user_input)
            print("AI:", response)
            text_to_speech(response)

        elif choice == "3":
            script = input("Enter your presentation script: ")
            feedback = evaluate_presentation(script)
            print("Feedback:", feedback)

        elif choice == "4":
            print("Training Modules:\n1. Impromptu Speaking\n2. Storytelling\n3. Conflict Resolution")
            mod_choice = input("Choose a module: ")
            module_map = {"1": "impromptu", "2": "storytelling", "3": "conflict_resolution"}
            if mod_choice in module_map:
                feedback = training_module(module_map[mod_choice])
                print("Training Feedback:", feedback)
            else:
                print("Invalid choice.")

        elif choice == "5":
            break
        else:
            print("Invalid choice. Try again.")

if __name__ == "__main__":
    main()


Options:
1. Chat with AI
2. Voice Input (May not work in all environments)
3. Evaluate Presentation
4. Training Modules
5. Exit
Choose an option: 1
Enter your message: hii
🔍 API Response: {'id': 'gen-1742056410-4wtR1hAd2qrpmWEc6CnS', 'provider': 'DeepInfra', 'model': 'mistralai/mistral-7b-instruct', 'object': 'chat.completion', 'created': 1742056410, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': " Hello! How can I assist you with verbal communication today? I'm here to help you improve your communication skills, whether you need help with public speaking, presentations, conversations, or making your messages more clear and impactful. Let's get started!", 'refusal': None}}], 'usage': {'prompt_tokens': 16, 'completion_tokens': 54, 'total_tokens': 70}}
AI:  Hello! How can I assist you with verbal communication today? I'm here to help you improve your communication skills, whether you need he

KeyboardInterrupt: Interrupted by user